In [2]:
import requests
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import copernicusmarine
import xarray as xr

In [3]:
copernicusmarine.subset(
  dataset_id="cmems_mod_bal_phy_anfc_PT1H-i",
  dataset_version="202411",
  variables=["thetao", "sla"],
  minimum_longitude=10.104315,
  maximum_longitude=10.297262,
  minimum_latitude=54.300509,
  maximum_latitude=54.496203,
  start_datetime="2023-10-01T12:00:00",
  end_datetime="2025-02-11T12:00:00",
  minimum_depth=0.5016462206840515,
  maximum_depth=0.5016462206840515,
  coordinates_selection_method="strict-inside",
  disable_progress_bar=False,
  username='mbanzhaf',
  password='6bF$ebvr',
  output_filename='subset_output.nc'
)

INFO - 2025-02-12T14:29:50Z - Selected dataset version: "202411"
INFO - 2025-02-12T14:29:50Z - Selected dataset part: "default"
INFO - 2025-02-12T14:29:55Z - Starting download. Please wait...
100%|██████████| 12/12 [00:01<00:00,  6.35it/s]
INFO - 2025-02-12T14:29:58Z - Successfully downloaded to subset_output.nc


ResponseSubset(file_path=PosixPath('subset_output.nc'), output_directory=PosixPath('.'), filename='subset_output.nc', file_size=7.679908396946565, data_transfer_size=68.52201526717558, variables=['thetao', 'sla'], coordinates_extent=[GeographicalExtent(minimum=10.124911308288574, maximum=10.291577339172363, unit='degrees_east', coordinate_id='longitude'), GeographicalExtent(minimum=54.30827713012695, maximum=54.491607666015625, unit='degrees_north', coordinate_id='latitude'), TimeExtent(minimum='2023-10-01T12:00:00+00:00', maximum='2025-02-11T12:00:00+00:00', unit='iso8601', coordinate_id='time'), GeographicalExtent(minimum=0.5016462206840515, maximum=0.5016462206840515, unit='m', coordinate_id='depth')], status='000', message='The request was successful.', file_status='DOWNLOADED')

In [4]:
file = 'subset_output.nc'

ds = xr.open_dataset(file)

# to dataframe
df = ds.to_dataframe().reset_index()



long = 10.152689
lat =  54.358276

val = 0.5
# Extract data which is close to the given coordinates
df_subset = df[
    (df['longitude'] >= long - val) & (df['longitude'] <= long + val) &
    (df['latitude'] >= lat - val) & (df['latitude'] <= lat + val)
]


df_subset = df_subset.dropna(axis=0, how='any')
df_subset = df_subset.groupby('time').agg({'sla': 'mean', 'thetao': 'mean'})
df_subset = df_subset.reset_index()
df_subset

,time,sla,thetao
0,2023-10-01 12:00:00,-0.100990,17.117083
1,2023-10-01 13:00:00,-0.080731,17.107500
2,2023-10-01 14:00:00,-0.059847,17.098394
3,2023-10-01 15:00:00,-0.050460,17.093483
4,2023-10-01 16:00:00,-0.043082,17.087259
...,...,...,...
11972,2025-02-11 08:00:00,0.081044,3.710460
11973,2025-02-11 09:00:00,0.073907,3.693938
11974,2025-02-11 10:00:00,0.072476,3.685151
11975,2025-02-11 11:00:00,0.076260,3.684210


In [5]:

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_subset['time'],
    y=df_subset['thetao'],
    mode='lines',
    name='Sea Water Temperature'
))

fig.update_layout(
    title='Sea Water Temperature near Kiel',
    xaxis_title='Time',
    yaxis_title='Sea Water Temperature (°C)',
    template='plotly_white'
)

fig.show()


In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_subset['time'],
    y=df_subset['sla'],
    mode='lines',
    name='Sea Water Temperature'
))

fig.update_layout(
    title='Sea Level Anomaly near Kiel',
    xaxis_title='Time',
    yaxis_title='Sea Level Anomaly (m)',
    template='plotly_white',
    #xaxis_type='category'  # Disable automatic averaging
)

fig.show()

In [7]:
from frost_server import FrostServer

server = FrostServer()
observation_url = server.get_observations_url()
content = server.get_content(observation_url)
all_observations = server.get_all_observations()

df_obs = pd.DataFrame(all_observations)
df_obs["phenomenonTime"] = pd.to_datetime(df_obs["phenomenonTime"])
df_obs["resultTime"] = pd.to_datetime(df_obs["resultTime"])

# convert result to float
df_obs["result"] = df_obs["result"].astype(float)

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_obs['phenomenonTime'],
    y=df_obs['result'],
    mode='lines',
    name='Temperature T-Box'
))

fig.add_trace(go.Scatter(
    x=df_subset['time'],
    y=df_subset['thetao'],
    mode='lines',
    name='Sea Water Temperature Copernicus'
))

fig.update_layout(
    title='Sea Water Temperature in Kiel',
    xaxis_title='Time',
    yaxis_title='Sea Water Temperature (°C)',
    template='plotly_white'
)

fig.show()

In [9]:
# calculate the average difference between the two datasets
diff = df_obs["result"].mean() - df_subset["thetao"].mean()
print(f"Average difference between the two datasets: {diff:.2f}°C")

Average difference between the two datasets: 3.34°C


In [10]:
# delte files which end with .nc
import os
import glob

files = glob.glob('*.nc')

for f in files:
    os.remove(f)
    print(f"Deleted {f}")


Deleted subset_output.nc
